### A. Librerias

In [1]:
import os
import time
import json
import requests

import pandas as pd

from utils import sheets
from bs4 import BeautifulSoup

### B. Funciones

In [2]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

def generar_url_google_maps(latitud, longitud):
    url = f"https://www.google.com/maps?q={latitud},{longitud}"
    return url

def generar_url_street_view(latitud, longitud):
    url = f"https://www.google.com/maps?q=&layer=c&cbll={latitud},{longitud}"
    return url

### C. Ejecución - Buscar departamentos por distritos

In [3]:
# 1. Listar los distritos y sus urls
distritos_urls = [
                  ['surquillo','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-surquillo-lima-lima-150141']
                , ['miraflores','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-miraflores-lima-lima-150122']
                , ['barranco','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-barranco-lima-lima-150104']
                , ['breña','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-bre%C3%B1a-lima-lima-150105']
                , ['jesus_maria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-jesus-maria-lima-lima-150113']
                , ['lince', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-lince-lima-lima-150116']
                , ['magdalena', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-magdalena-del-mar-lima-lima-150120']
                , ['pueblo_libre', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-pueblo-libre-lima-lima-150121']
                , ['san_isidro', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-isidro-lima-lima-150131']
                , ['san_miguel', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['la_victoria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['san_borja', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-en-san-borja-lima-lima-150130']
                , ['chorrillos', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-en-chorrillos-lima-lima-150108']
                , ['ate', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-en-ate-lima-lima-150103']
            ]

In [4]:
# 2. Extraer los datos de cada distrito
for distrito, url in distritos_urls:
    # 1. Obtener la URL por request
    response = requests.get(url)
    print(response.status_code, '-> ', distrito)

    if response.status_code == 200:
        # 2. Parsear el HTML
        html = response.text

        # 3. Guardar el HTML en un archivo
        with open(os.path.join(os.getcwd(),'originals', distrito + '.html'), 'w') as outfile:
            outfile.write(html)

        # 4. Tiempo de espera
        time.sleep(2)

200 ->  san_borja
200 ->  chorrillos
200 ->  ate


In [5]:
# 3. Listar los proyectos y sus urls
urls_list = []
for distrito, x in distritos_urls:
    # A. Leer el html
    with open(os.path.join(os.getcwd(),'originals', distrito+ '.html'), 'r') as infile:
        lines = infile.readlines()

    # B. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # C. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # D. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # E. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append(base)

len(urls_list)
    

50

In [6]:
# 4. Extraer los datos de cada proyecto
i = 0
modelos_list = []
dataframes_list = []
for url in urls_list:
    # A. Obtener la URL por request
    response = requests.get(url)
    # print(response.status_code,':' ,url)

    if response.status_code == 200:
        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        #### A. Datos generales
        # Extrae el texto de <h1>Eres Fase 2</h1>
        try:
            proyecto = soup.find('h1').get_text(strip=True)
        except:
            proyecto = '-'
        # print(f'Proyecto: {proyecto}')

        # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
        try:
            direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
        except:
            direccion = '-'
        # print(f'Dirección: {direccion}')

        # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
        try:
            distrito = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
        except:
            distrito = '-'
        # print(f'Distrito: {distrito}')

        # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
        try:
            precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
        except:
            precio_desde = '-'
        # print(f'Precio desde: {precio_desde}')

        # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
        try:
            valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
        except:
            valor_referencia = '-'
        # print(f'Valor de referencia: {valor_referencia}')

        # Listar las áreas comunes
        try:
            areas_lista = soup.find('ul', class_='Project-areas-list')
            elementos_lista = areas_lista.find_all('li')
            areas_comunes = ', '.join(elemento.get_text(strip=True) for elemento in elementos_lista)
        except:
            areas_comunes = '-'

        # Encuentra el campo de entrada para la latitud
        try:
            input_latitud = soup.find('input', id='latitude')
            latitud = input_latitud['value']
        except:
            latitud = '-'
        # print(f'Latitud: {latitud}')

        try:
            input_longitud = soup.find('input', id='longitude')
            longitud = input_longitud['value']
        except:
            longitud = '-'
        # print(f'Longitud: {longitud}')

        # print('')
        # print('Datos de la inmobiliaria')
        #### B. Datos de la inmobiliaria
        try:
            # Encuentra el div con la clase "bx-data-project box-st"
            div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

            if div_informacion_proyecto:
                # Encuentra todas las filas de la tabla dentro del div
                filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

                # Itera sobre las filas de la tabla e imprime la información
                for fila in filas_tabla:
                    columnas = fila.find_all('td')
                    if len(columnas) == 2:
                        try:
                            etiqueta = columnas[0].strong.get_text(strip=True)
                            valor = columnas[1].get_text(strip=True)
                            
                            if etiqueta == 'Tipo de inmueble':
                                tipo_inmueble = valor
                            elif etiqueta == 'Área total':
                                area_total = valor
                            elif etiqueta == 'Dormitorios':
                                dormitorios = valor
                            elif etiqueta == 'Etapa del proyecto':
                                etapa_proyecto = valor
                            elif etiqueta == 'Fecha de entrega':
                                fecha_entrega = valor
                            elif etiqueta == 'Financiamiento':
                                financiamiento = valor
                            elif etiqueta == 'Inmobiliaria Nombre':
                                inmobiliaria_nombre = valor
                            
                            # print(f'{etiqueta} -> {valor}')

                        except:
                            pass

                # También puedes extraer información específica usando el método find para elementos específicos
                # Por ejemplo, para obtener la información de la inmobiliaria
                inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)
                # print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')
        except:
            tipo_inmueble = '-'
            area_total = '-'
            dormitorios = '-'
            etapa_proyecto = '-'
            fecha_entrega = '-'
            financiamiento = '-'
            inmobiliaria_nombre = '-'
            # print('No se encontró información de la inmobiliaria')

        #### C. Modelos disponibles
        try:
            modelos_disponibles = soup.find_all('div', class_='Project-available-model')
            modelos_list.append([proyecto,modelos_disponibles, url])
            # print('')
            # print('Modelos disponibles')
            # print(f'Cantidad de modelos: {len(modelos_disponibles)}')
        except:
            pass

        #### D. Unir los datos en un diccionario
        proyecto_data = {
            'Proyecto': proyecto,
            'Dirección': direccion,
            'Distrito': distrito,
            'Precio Desde': precio_desde,
            'Valor de Referencia': valor_referencia,
            'Latitud': latitud,
            'Longitud': longitud,
            'Tipo de Inmueble': tipo_inmueble,
            'Área Total': area_total,
            'Dormitorios': dormitorios,
            'Etapa del Proyecto': etapa_proyecto,
            'Fecha de Entrega': fecha_entrega,
            'Financiamiento': financiamiento,
            'Inmobiliaria Nombre': inmobiliaria_nombre,
            'Áreas Comunes': areas_comunes,
            'URL Nexoinmobiliario': url
        }

        # Convierte el diccionario en un DataFrame
        df = pd.DataFrame([proyecto_data])
        dataframes_list.append(df)

        # Incrementa el contador
        print(i, '-> ', proyecto , '-> ', url)
        i += 1
        
        # Tiempo de espera
        time.sleep(2)

0 ->  Millet 173 ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3080-millet-173
1 ->  Niza ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3023-niza
2 ->  ORDOÑEZ ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2616-ordonez
3 ->  DALÍ PLACE ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2687-dali-place
4 ->  PARQUE EULER ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2175-parque-euler
5 ->  TORRE TALE PRIMAVERA - ETAPA I ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2877-torre-tale-primavera-etapa-i
6 ->  Esquivel101 ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1902-esquivel101
7 ->  Hometown Condominio ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3167-hometown-condominio
8 ->  Sena ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3173-sena
9 ->  Toscanini ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2985-toscanini
10 ->  Roo

In [7]:
# 1. Concatenar los DataFrames
df_proyectos = pd.concat(dataframes_list)

# 2. Crear columnas de Google Maps y Street View
df_proyectos['Google Maps'] = df_proyectos.apply(lambda row: generar_url_google_maps(row['Latitud'], row['Longitud']), axis=1)
df_proyectos['Street View'] = df_proyectos.apply(lambda row: generar_url_street_view(row['Latitud'], row['Longitud']), axis=1)

# Eliminar las columnas de Latitud y Longitud
df_proyectos = df_proyectos.drop(columns=['Latitud', 'Longitud'])
df_proyectos

,Proyecto,Dirección,Distrito,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,Google Maps,Street View
0,Millet 173,Jr. Millet 173,San Borja San Borja,"S/. 653,536","$ 173,952",Departamento,80.10 a 179.80 m2,3 a 3,En planos,"31 de Diciembre, 2024",Banco BCP,GRUPO BA,"Lobby, Otros",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.090520788609...,https://www.google.com/maps?q=&layer=c&cbll=-1...
0,Niza,Av. San Borja Norte. 232,San Borja San Borja,"S/. 420,000","$ 111,791",Departamento,49.60 a 90.84 m2,2 a 3,En planos,"31 de Marzo, 2025",Banco BCP,EE INMOBILIARIA,Lobby,https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0952263,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,ORDOÑEZ,Jr. Eduardo Ordoñez 187-189,Santo Tomas San Borja,"S/. 676,545","$ 180,076",Departamento,90.47 a 192.79 m2,3 a 3,Entrega inmediata,"30 de Noviembre, 2023",Banco Interbank,BALANCE GRUPO INMOBILIARIO,Lobby,https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.1056923,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,DALÍ PLACE,Av. San Borja Norte 1023 San Borja,San Borja San Borja,"S/. 388,000","$ 103,274",Departamento,40.79 a 74.94 m2,1 a 3,En construcción,"01 de Octubre, 2024",BBVA,NOI INMOBILIARIA,"Lobby, Sala de Internet, Zona de Parrillas, Otros",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.092666158417...,https://www.google.com/maps?q=&layer=c&cbll=-1...
0,PARQUE EULER,Jirón Leonhard Euler N°204 esquina con Calle N...,San Borja San Borja,"S/. 778,000","$ 207,080",Departamento,90.00 a 90.00 m2,2 a 2,Entrega inmediata,"30 de Septiembre, 2022",BBVA,BALANCE GRUPO INMOBILIARIO,"Jardin interior, Lobby, Video vigilancia",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.102288181225...,https://www.google.com/maps?q=&layer=c&cbll=-1...
0,TORRE TALE PRIMAVERA - ETAPA I,"Av. San Luis esquina Av. Angamos Este, esquina...",San Borja,"S/. 296,000","$ 78,786",Departamento,37.00 a 115.00 m2,1 a 3,En planos,"31 de Mayo, 2026",BBVA,TALE INMOBILIARIA,"Gimnasio, Lobby, Piscina, Sala Bar, Sala de Ni...",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.110887994060...,https://www.google.com/maps?q=&layer=c&cbll=-1...
0,Esquivel101,Esquivel 101,San Borja San Borja,"S/. 857,500","$ 228,241",Departamento,96.88 a 130.38 m2,3 a 3,Entrega inmediata,"30 de Abril, 2023",Banco Interbank,RB ARQUITECTURA,"Lobby, Sala de usos Múltiples, Video vigilanci...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0948611,-76....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,Hometown Condominio,Durero 490 san borja,San Borja,"$ 196,000","S/. 736,372",Departamento,89.77 a 253.94 m2,2 a 4,En planos,"30 de Mayo, 2025",BBVA,ESPARQ INVERSIONES,"Área de juegos para niños, Areas verdes, Gimna...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0940626,-76....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,Sena,"Av. San Borja Sur 546, San Borja",San Borja,"$ 185,000","S/. 695,045",Departamento,90.01 a 184.82 m2,3 a 4,En planos,"31 de Diciembre, 2024",Banco BCP,EE INMOBILIARIA,Lobby,https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.1013995,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,Toscanini,CALLE TOSCANINI 129,San Borja,"S/. 766,400","$ 203,993",Departamento,90.74 a 194.94 m2,2 a 3,En planos,"31 de Julio, 2024",Otros,RB ARQUITECTURA,Lobby,https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.107221696946...,https://www.google.com/maps?q=&layer=c&cbll=-1...


### D. Ejecución - Buscar detalle de departamentos

In [8]:
data_modelos = []
for proyecto, modelos_disponibles, url in modelos_list:
    for modelo in modelos_disponibles:
        try:
            # Extrae información del enlace de imagen
            #imagen_enlace = modelo.find('a', class_='popup-link')['data-links'].strip()
            #print(f'Enlace de la imagen: {imagen_enlace}')

            # Extrae información del modelo
            nombre_modelo = modelo.find('span', class_='name_tipology').get_text(strip=True).strip()
            #print(f'Nombre del modelo: {nombre_modelo}')

            # Extrae información del número de pisos
            num_pisos = modelo.find('span', class_='num_pisos').get_text(strip=True).strip()
            #print(f'Número de pisos: {num_pisos}')

            # Extrae información del dormitorio, área y precio
            dormitorio = modelo.find('span', class_='bedroom').get_text(strip=True).strip()
            area = modelo.find('span', class_='area').get_text(strip=True).strip()
            precio = modelo.find('span', class_='price').get_text(strip=True).strip()

            #print(f'Dormitorio: {dormitorio}')
            #print(f'Área: {area}')
            #print(f'Precio desde: {precio}')

            # También puedes extraer información del enlace de cotización
            #enlace_cotizacion = modelo.find('a', class_='Btn-ver--red')['href'].strip()
            #print(f'Enlace de cotización: {enlace_cotizacion}')

            # Agregar datos del modelo a la lista
            data_modelos.append({
                'Proyecto': proyecto,
                'Nombre del modelo': nombre_modelo,
                'Número de pisos': num_pisos,
                'Dormitorio': dormitorio,
                'Área': area,
                'Precio desde': precio,
                'URL Nexoinmobiliario': url
            })

            #print("\n---\n")  # Separador entre modelos
        except:
            pass

df_modelos = pd.DataFrame(data_modelos)
df_modelos

,Proyecto,Nombre del modelo,Número de pisos,Dormitorio,Área,Precio desde,URL Nexoinmobiliario
0,Millet 173,TIPO A,Piso: 1,3,138.25 ...,"S/ 1,047,643",https://nexoinmobiliario.pe/proyecto/venta-de-...
1,Millet 173,TIPO B,Piso: 1,3,137.35 ...,"S/ 897,525",https://nexoinmobiliario.pe/proyecto/venta-de-...
2,Millet 173,TIPO C,"Pisos: 2, 3, 4",3,90.25 ...,"S/ 775,157",https://nexoinmobiliario.pe/proyecto/venta-de-...
3,Millet 173,TIPO D,"Pisos: 2, 3, 4",3,90.10 ...,"S/ 773,869",https://nexoinmobiliario.pe/proyecto/venta-de-...
4,Millet 173,TIPO E,"Pisos: 2, 3, 4",3,80.10 ...,"S/ 653,536",https://nexoinmobiliario.pe/proyecto/venta-de-...
...,...,...,...,...,...,...,...
459,Madrid Santa Clara III,FLAT TIPO 4 TORRE I,"Pisos: 5, 6, 9, 10",3,60.65 ...,"S/ 220,000",https://nexoinmobiliario.pe/proyecto/venta-de-...
460,Madrid Santa Clara III,FLAT TIPO 1 TORRE I,"Pisos: 8, 9, 10",3,61.17 ...,"S/ 219,000",https://nexoinmobiliario.pe/proyecto/venta-de-...
461,Madrid Santa Clara III,FLAT TIPO 4 TORRE K,"Pisos: 6, 7, 10",3,61.17 ...,"S/ 225,000",https://nexoinmobiliario.pe/proyecto/venta-de-...
462,Madrid Santa Clara III,FLAT TIPO 4 TORRE J,"Pisos: 8, 9, 10",3,62.60 ...,"S/ 234,000",https://nexoinmobiliario.pe/proyecto/venta-de-...


### E. Dar formato a df_proyectos

In [9]:
def formato_fecha(mes_str):
    meses = {
        'Enero': '01',
        'Febrero': '02',
        'Marzo': '03',
        'Abril': '04',
        'Mayo': '05',
        'Junio': '06',
        'Julio': '07',
        'Agosto': '08',
        'Septiembre': '09',
        'Octubre': '10',
        'Noviembre': '11',
        'Diciembre': '12'
    }

    mes_split = mes_str.split(' ')

    dia = mes_split[0]
    mes = meses.get(mes_split[2].replace(',',''))
    anio = mes_split[3]

    mes_format = f'{anio}-{mes}-{dia}'
    if mes_format:
        return mes_format
    else:
        return None

In [10]:
# Guardar df_proyectos y df_modelos en un parquet
df_proyectos.to_parquet(os.path.join(os.getcwd(),'outputs','df_proyectos.parquet'))
df_modelos.to_parquet(os.path.join(os.getcwd(),'outputs','df_modelos.parquet'))

# Guardar df_proyectos y df_modelos en excel
df_proyectos.to_excel(os.path.join(os.getcwd(),'outputs','df_proyectos.xlsx'), index=False)
df_modelos.to_excel(os.path.join(os.getcwd(),'outputs','df_modelos.xlsx'), index=False)

In [11]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_proyectos_format = df_proyectos.copy()
df_proyectos_format['Fecha de Entrega'] = df_proyectos_format['Fecha de Entrega'].apply(formato_fecha)
df_proyectos_format['Fecha de Entrega'] = pd.to_datetime(df_proyectos_format['Fecha de Entrega'])

# 2. Replace a las columnas de 'Precio Desde' .replace('S/.','').replace(',','') y 'Valor de Referencia' .replace('$','').replace(',','')
df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].str.replace('S/. ','').str.replace(',','').replace('-','0')
df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].str.replace('$ ','').str.replace(',','').replace('-','0')

# 3. Castear las columnas de 'Precio Desde' y 'Valor de Referencia' a int
# df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].astype(int)
# df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].astype(int)

# 4. Renombrar columna 'Precio Desde':'Precio Desde (S/)' , 'Valor de Referencia':'Valor de Referencia ($)'
# df_proyectos_format = df_proyectos_format.rename(columns={'Precio Desde':'Precio Desde (S/.)', 'Valor de Referencia':'Valor de Referencia ($)'})

# 5. Hacer un upper a las columas que son string
df_proyectos_format['Proyecto'] = df_proyectos_format['Proyecto'].str.upper()
df_proyectos_format['Dirección'] = df_proyectos_format['Dirección'].str.upper()
df_proyectos_format['Financiamiento'] = df_proyectos_format['Financiamiento'].str.upper()
df_proyectos_format['Inmobiliaria Nombre'] = df_proyectos_format['Inmobiliaria Nombre'].str.upper()

# 6. Reordenar las columnas
df_proyectos_format = df_proyectos_format[['Proyecto', 'Distrito', 'Precio Desde', 'Valor de Referencia', 'Tipo de Inmueble', 'Área Total', 'Dormitorios', 'Etapa del Proyecto', 'Fecha de Entrega', 'Financiamiento', 'Inmobiliaria Nombre', 'Áreas Comunes', 'Dirección', 'Google Maps', 'Street View', 'URL Nexoinmobiliario']]

df_proyectos_format.head(3)

,Proyecto,Distrito,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,Dirección,Google Maps,Street View,URL Nexoinmobiliario
0,MILLET 173,San Borja San Borja,653536,173952,Departamento,80.10 a 179.80 m2,3 a 3,En planos,2024-12-31,BANCO BCP,GRUPO BA,"Lobby, Otros",JR. MILLET 173,https://www.google.com/maps?q=-12.090520788609...,https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...
0,NIZA,San Borja San Borja,420000,111791,Departamento,49.60 a 90.84 m2,2 a 3,En planos,2025-03-31,BANCO BCP,EE INMOBILIARIA,Lobby,AV. SAN BORJA NORTE. 232,"https://www.google.com/maps?q=-12.0952263,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...
0,ORDOÑEZ,Santo Tomas San Borja,676545,180076,Departamento,90.47 a 192.79 m2,3 a 3,Entrega inmediata,2023-11-30,BANCO INTERBANK,BALANCE GRUPO INMOBILIARIO,Lobby,JR. EDUARDO ORDOÑEZ 187-189,"https://www.google.com/maps?q=-12.1056923,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...


In [12]:
df_proyectos_format.dtypes

Proyecto                        object
Distrito                        object
Precio Desde                    object
Valor de Referencia             object
Tipo de Inmueble                object
Área Total                      object
Dormitorios                     object
Etapa del Proyecto              object
Fecha de Entrega        datetime64[ns]
Financiamiento                  object
Inmobiliaria Nombre             object
Áreas Comunes                   object
Dirección                       object
Google Maps                     object
Street View                     object
URL Nexoinmobiliario            object
dtype: object

### F. Dar formato a df_modelos

In [13]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_modelos_format = df_modelos.copy()

# 2. Replace a las columnas de 'Precio Desde' .replace('S/.','').replace(',','') y 'Valor de Referencia' .replace('$','').replace(',','')
# df_modelos_format['Precio desde'] = df_modelos_format['Precio desde'].str.replace('S/ ','').str.replace(',','')
# df_modelos_format['Precio desde'] = df_modelos_format['Precio desde'].astype(int)

# 3. Hacer un upper a las columas que son string
df_modelos_format['Proyecto'] = df_modelos_format['Proyecto'].str.upper()
df_modelos_format['Nombre del modelo'] = df_modelos_format['Nombre del modelo'].str.upper()

# 4. Hacer .split(' ')[0] a la columna 'Área'
df_modelos_format['Área'] = df_modelos_format['Área'].apply(lambda x: x.split(' ')[0])

# 5. Renombrar columna 'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'
df_modelos_format = df_modelos_format.rename(columns={'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'})

df_modelos_format.head(3)

,Proyecto,Nombre del modelo,Piso del departamento,Dormitorios,Área (m2),Precio desde,URL Nexoinmobiliario
0,MILLET 173,TIPO A,Piso: 1,3,138.25,"S/ 1,047,643",https://nexoinmobiliario.pe/proyecto/venta-de-...
1,MILLET 173,TIPO B,Piso: 1,3,137.35,"S/ 897,525",https://nexoinmobiliario.pe/proyecto/venta-de-...
2,MILLET 173,TIPO C,"Pisos: 2, 3, 4",3,90.25,"S/ 775,157",https://nexoinmobiliario.pe/proyecto/venta-de-...


In [14]:
df_modelos_format.dtypes

Proyecto                 object
Nombre del modelo        object
Piso del departamento    object
Dormitorios              object
Área (m2)                object
Precio desde             object
URL Nexoinmobiliario     object
dtype: object

### G. Guardar en google sheets 
drive-sheets-service@molten-topic-396921.iam.gserviceaccount.com  
https://docs.google.com/spreadsheets/d/1PDphDA-IHCnuykZoDWjzkl_f0AxOsw0bUsIbpE3weZg/edit#gid=0


In [15]:
# 1. Crear flujo de lectura y escritura
sh = sheets.autentificar(os.path.join(os.getcwd(),'utils', 'keys_sheets.json'), '1PDphDA-IHCnuykZoDWjzkl_f0AxOsw0bUsIbpE3weZg','id')

# 2. Guardar df_proyectos_format en la hoja 'Proyectos'
sheets.dataframe_to_sheet(sh, df_proyectos_format, 'Proyectos')

# 3. Guardar df_modelos_format en la hoja 'Detalles'
sheets.dataframe_to_sheet(sh, df_modelos_format, 'Detalles')

In [16]:
print('Se guardaron los datos en Google Sheets')

Se guardaron los datos en Google Sheets
